In [1]:
import ee
import geemap

Map = geemap.Map()

countries = ee.FeatureCollection("FAO/GAUL/2015/level1")
thessalia = countries.filter(ee.Filter.eq('ADM1_NAME', 'Thessalia'))

# Functions
def calculate_ndwi(image):
    ndwi = image.normalizedDifference(["B8", "B3"])
    return ndwi

# Create a binary mask image where masked areas are set to 1
mask_image = ee.Image(1).clip(thessalia).mask()

## Before the event
sentinel2_before = ee.ImageCollection('COPERNICUS/S2')\
    .filterDate('2023-08-31', '2023-09-01').filterBounds(thessalia)

## After the event
sentinel2_after = ee.ImageCollection('COPERNICUS/S2')\
    .filterDate('2023-09-10', '2023-09-11').filterBounds(thessalia)

# Applying the NDWI index
ndwi_collection_before = sentinel2_before.map(calculate_ndwi)

# Applying the NDWI index
ndwi_collection_after = sentinel2_after.map(calculate_ndwi)

# Update the mask of NDWI collections using the binary mask
ndwi_collection_before_masked = ndwi_collection_before.map(lambda image: image.updateMask(mask_image))
ndwi_collection_after_masked = ndwi_collection_after.map(lambda image: image.updateMask(mask_image))

ndwi_vis = {"min": -1, "max": 1, "palette": ["green", "white", "blue"]}

left_layer = geemap.ee_tile_layer(ndwi_collection_before_masked, ndwi_vis, name='NDWI before')
right_layer = geemap.ee_tile_layer(ndwi_collection_after_masked, ndwi_vis, name='NDWI after')

Map.split_map(left_layer, right_layer, left_label='Before the event',
    right_label='After the event')
Map.setCenter(22, 38, 7)
Map

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=KhewTZShJZGWsNKiGpWm5TR06gFYOA-FvelMJoGVmCA&tc=1dbJxql2qkdZxbL7zeDfwRFC3yLJ53SS33uUlb_EoXg&cc=Z56sRc-GTpnLYyCq3j3JSVm9kZVedfL_uTHD2I68zFc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXm6FNWlGhFs_-GA3ek3JGJ0rgCw2FHHXeS2kWWJ__4k2e_Z7dVyWCQ

Successfully saved authorization token.


Map(center=[38, 22], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…